In [2]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from animal_shelter import AnimalShelter

# Rescue Type preferences
RESCUE_PREFS = {
    "Water Rescue": {
        "breeds": ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"],
        "sex": "Intact Female",
        "age_min": 26,   # weeks
        "age_max": 156,
    },
    "Mountain or Wilderness Rescue": {
        "breeds": ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiler"],
        "sex": "Intact Male",
        "age_min": 26,
        "age_max": 156,
    },
    "Disaster or Individual Tracking": {
        "breeds": ["Doberman Pinscher", "German Shepherd", "Golden Retriever", "Bloodhound", "Rottweiler"],
        "sex": "Intact Male",
        "age_min": 20,
        "age_max": 300,
    },
}
RESCUE_TYPES = list(RESCUE_PREFS.keys()) + ["Reset"]

###########################
# Data Manipulation / Model
###########################
# Username and password and CRUD Python module name

username = "aacuser"
password = "SNHU1234"

# Connect to database via CRUD Module
db = AnimalShelter(username, password)

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will return a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)


#########################
# Dashboard Layout / View
#########################
breed_options = (
    [{'label': b, 'value': b} for b in sorted(df['breed'].dropna().unique())]
    if 'breed' in df.columns else []
)
sex_options = (
    [{'label': s, 'value': s} for s in sorted(df['sex_upon_outcome'].dropna().unique())]
    if 'sex_upon_outcome' in df.columns else []
)
app = JupyterDash(__name__)

image_filename = 'gs-logo.png' # Grazioso Salvare Logo
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

app.layout = html.Div([
#    html.Div(id='hidden-div', style={'display':'none'}),
    html.Div(
        [
            html.Img(
                src=f'data:image/png;base64,{encoded_image.decode()}',
                style={'maxHeight': '80px', 'display': 'block', 'margin': '0 auto'}
            ),
            html.H1("CS-340 Dashboard - Chris O'Dell")
        ],
        style={'textAlign': 'center', 'marginBottom': '8px'}
    ),
    # Radio filter options
    html.Div([
        html.Label("Rescue Type:"),
        dcc.RadioItems(
            id='radio-rescue-type',
            options=[{'label': r, 'value': r} for r in RESCUE_TYPES],
            value='Reset',  # start unfiltered
            labelStyle={'display': 'inline-block', 'marginRight': '12px'},
            persistence=True, persistence_type='session'
        ),

        html.Br(),
    # Dropdown filter options
        html.Label("Select Breed:"),
        dcc.Dropdown(
            id='dropdown-breed',
            options=breed_options,        # will be overridden when a rescue type is chosen
            placeholder="Filter by breed",
            multi=False,
            clearable=True,
            persistence=True, persistence_type='session'
        ),
    ], style={'maxWidth': '900px', 'margin': '0 auto'}),

    html.Hr(),
    # Interactive data table
    dash_table.DataTable(id='datatable-id',
                         columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
                         data=df.to_dict('records'),
                         filter_action="native",
                         sort_action="native",
                         sort_mode="multi",
                         page_action="native",
                         page_current=0,
                         page_size=10,
                         # Single row selection
                         row_selectable="single",
                         selected_rows=[0],
                         style_table={'overflowX': 'auto', 'minWidth': '100%'},
                         style_cell={
                             'textAlign': 'left',
                             'padding': '6px',
                             'whiteSpace': 'normal',
                             'height': 'auto',
                             'fontFamily': 'Arial',
                             'fontSize': 12,
                         },
                         style_header={'fontWeight': 'bold'},
                        ),
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################
    
# Build Mongo query from UI controls
def build_query(rescue_type, breed):
    # Always limit to dogs for these rescue filters
    query = {'animal_type': 'Dog'}

    if rescue_type and rescue_type != "Reset":
        prefs = RESCUE_PREFS.get(rescue_type, {})
        breeds = prefs.get('breeds', [])
        preferred_sex = prefs.get('sex')
        age_min = prefs.get('age_min')
        age_max = prefs.get('age_max')

        # Breed logic: if a specific breed is selected and it belongs to the list, use it,
        # otherwise allow any of the preferred breeds.
        if breed and (not breeds or breed in breeds):
            query['breed'] = breed
        elif breeds:
            query['breed'] = {'$in': breeds}
        # Sex preference
        if preferred_sex:
            query['sex_upon_outcome'] = preferred_sex
        # Age range (inclusive)
        if age_min is not None and age_max is not None:
            query['age_upon_outcome_in_weeks'] = {'$gte': age_min, '$lte': age_max}
    else:
        # Reset / unfiltered; allow the user to optionally pick a breed
        if breed:
            query['breed'] = breed
        # No sex or age filter applied in Reset mode

    return query

# Update data table from Mongo
@app.callback(
    Output('datatable-id', 'data'),
    [
        Input('radio-rescue-type', 'value'),
        Input('dropdown-breed', 'value'),
    ]
)
def update_dashboard(rescue_type, breed):
    query = build_query(rescue_type, breed)
    try:
        docs = db.read(query)  # server-side filtering via Mongo
    except Exception as e:
        print(f"Mongo read error: {e}")
        return []

    if not docs:
        dff = pd.DataFrame(columns=df.columns)
    else:
        dff = pd.DataFrame.from_records(docs)
        dff.drop(columns=['_id'], errors='ignore', inplace=True)
        # ensure all DataTable columns exist and in order
        for col in df.columns:
            if col not in dff.columns:
                dff[col] = None
        dff = dff[df.columns]

    return dff.to_dict('records')
# Display the breeds of animal based on quantity represented in
# the data table
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")])
def update_graphs(viewData):
    dff = pd.DataFrame(viewData) if viewData is not None else df.copy()
    if dff.empty or 'breed' not in dff.columns:
        return html.Div("No data available for chart.", style={'padding': '12px'})

    counts = (
        dff['breed']
        .fillna('Unknown')
        .value_counts()
        .reset_index()
        .rename(columns={'index': 'breed', 'breed': 'count'})
    )

    fig = px.pie(
        counts,
        names='breed',
        values='count',
        title='Breed Distribution'
    )
    fig.update_traces(
        textposition='inside',
        texttemplate='%{percent:.1%}',
        hovertemplate='%{label}<br>Count=%{value}<br>%{percent}'
    )
    fig.update_layout(margin=dict(l=10, r=10, t=40, b=10))

    return dcc.Graph(figure=fig)
    
#This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    if not selected_columns:
        return[]
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):  
    if viewData is None:
        return
    elif index is None:
        return
    
    dff = pd.DataFrame.from_dict(viewData)
    # Because we only allow single row selection, the list can be converted to a row index here
    if index is None:
        row = 0
    else: 
        row = index[0]
        
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            # Column 13 and 14 define the grid-coordinates for the map
            # Column 4 defines the breed for the animal
            # Column 9 defines the name of the animal
            dl.Marker(position=[dff.iloc[row,13],dff.iloc[row,14]], children=[
                dl.Tooltip(dff.iloc[row,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[row,9])
                ])
            ])
        ])
    ]



app.run_server(debug=True)


Dash app running on http://127.0.0.1:14011/
